In [1]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Read the CSV File
csv_file = r'C:\Users\Wolfrank\Desktop\sdf.csv'  # Use a raw string for the file path
df = pd.read_csv(csv_file)

# Step 2: Create a SQLite Database
conn = sqlite3.connect('trades.db')  # Replace with your desired database name

# Step 3: Write the DataFrame to the Database
table_name = 'trades_table'  # Replace with your desired table name
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Sample SQL queries

# 1. Retrieve all rows from the trades_table
query = f"SELECT * FROM {table_name}"
df_all_trades = pd.read_sql_query(query, conn)


In [7]:

# 2. Calculate the average profit per trade. 
query = f"SELECT AVG(Strat_Profits) AS average_profits FROM {table_name}"
average_profits = pd.read_sql_query(query, conn)['average_profits'].iloc[0].round(2)

# Display
print("The average  of profits is:", average_profits)

The average  of profits is: 953.47


In [8]:

# 3. Find trades with a profit (Change > 0)
query = f"SELECT * FROM {table_name} WHERE Change > 0"
df_profitable_trades = pd.read_sql_query(query, conn)

print('The trades with a profit are ', df_profitable_trades)

The trades with a profit are      Tradeno        Date  Shares Symbol   Open   High    Low  Close     Volume  \
0         4  2020-07-09    1000   TQQQ  28.85  29.17  27.47  28.92  158798000   
1        11  2020-07-20    1000   TQQQ  28.20  30.56  27.77  30.34  125810800   
2        20  2020-07-31    1000   TQQQ  29.83  29.88  28.22  29.84  179907600   
3        60  2020-09-28    1000   TQQQ  32.32  32.50  31.39  32.43  136665600   
4        87  2020-11-04    1000   TQQQ  33.90  35.66  33.33  35.02  169187600   
5       115  2020-12-15    1000   TQQQ  42.15  42.50  41.23  42.47   78708800   
6       131  2021-01-08    1000   TQQQ  46.83  47.72  45.85  47.57   76407600   
7       138  2021-01-20    1000   TQQQ  47.93  49.94  47.76  49.49   60868000   
8       147  2021-02-02    1000   TQQQ  49.94  51.48  49.88  50.91   46373200   
9       173  2021-03-11    1000   TQQQ  44.51  46.13  44.07  45.39   79467800   
10      187  2021-03-31    1000   TQQQ  44.40  46.30  44.26  45.63   80984400  

In [ ]:

# 4. Calculate the total volume for each symbol
query = f"SELECT Symbol, SUM(Volume) AS total_volume FROM {table_name} GROUP BY Symbol"
df_total_volume = pd.read_sql_query(query, conn)


In [ ]:

# 5. Find trades with a loss (Change < 0)
query = f"SELECT * FROM {table_name} WHERE Change < 0"
df_loss_trades = pd.read_sql_query(query, conn)


In [ ]:

# Data Visualization using Seaborn

# 1. Distribution of Shares Traded
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.histplot(df_all_trades['Shares'], bins=20, kde=True)
plt.xlabel('Shares Traded')
plt.title('Distribution of Shares Traded')
plt.show()

# 2. Profitable vs. Loss Trades (Count)
plt.figure(figsize=(18, 16))
sns.countplot(x='Win', data=df_all_trades, palette='viridis')
plt.xlabel('Trade Profit Amount ')
plt.title('Profitable Trades: Count')
plt.xticks(rotation=45)
plt.show()



# 2a. Profitable vs. Loss Trades (Count)
plt.figure(figsize=(18, 16))
sns.countplot(x='Loss', data=df_all_trades, palette='viridis')
plt.xlabel('Trade Loss Amount ')
plt.title('Losing Trades: Count')
plt.xticks(rotation=45)
plt.show()


# 3. Total Volume by Symbol (Bar Plot)
plt.figure(figsize=(10, 6))
sns.barplot(x='Symbol', y='total_volume', data=df_total_volume, palette='Set3')
plt.xlabel('Symbol')
plt.ylabel('Total Volume')
plt.title('Total Volume by Symbol')
plt.xticks(rotation=45)
plt.show()

# 4. Pairwise Relationships (Scatter Plot)
sns.pairplot(df_all_trades[['Shares', 'Open', 'High', 'Low', 'Close']])
plt.title('Pairwise Relationships')
plt.show()

# 5. Box Plot of Daily Excess Returns
plt.figure(figsize=(8, 6))
sns.boxplot(x='Excess_Return', data=df_all_trades, orient='vertical', palette='Pastel1')
plt.xlabel('Daily Excess Returns')
plt.title('Box Plot of Daily Excess Returns')
plt.show()

# Displaying the results
print("Sample Trades Data:")
print(df_all_trades)

print("\nAverage Shares Traded:", average_shares)

print("\nProfitable Trades:")
print(df_profitable_trades)

print("\nTotal Volume by Symbol:")
print(df_total_volume)

print("\nLoss Trades:")
print(df_loss_trades)

# Step 4: Close the Database Connection
conn.close()